In [3]:
# -*- coding: utf-8 -*-
"""
Created on Fri Oct 28 13:54:29 2016
Python3 update v0 Tue Mar 27 2018
Python3 update v2 Thu Aug 2 2018
update v3 Fri Apr 3 2020 - added error trapping and (hopefully) informative error messages. 

Requires:
    Thorlabs APT server dll installed in default location and configured for attached device.
    thorlabs_apt package (0.2 tested) installed as requirement of level2labs
    
@author: Aidan Hindmarch
"""

from os import environ, pathsep
from platform import system

if not system() == 'Windows':
    raise RuntimeError('Thorlabs APT device drivers only supported under Windows OS.')

termpath = environ['PATH']
if not r'Thorlabs\APT\APT Server' in termpath:
    environ['PATH'] = r'C:\Program Files (x86)\Thorlabs\APT\APT Server' + pathsep + r'C:\Program Files\Thorlabs\APT\APT Server' + pathsep + termpath
# Prepend default location of thorlabs apt server apt.dll to local PATH for terminal.
# This is required so that when we import thorlabs_apt module the _load_library
# function which automatically runs in core.py is able to find the system APT.dll by
# filename = ctypes.util.find_library("APT") - which searches the PATH.

try:
    from thorlabs_apt import core as apt_rs
except Exception as e:
    raise RuntimeError('Importing thorlabs_apt package failed (likely related to locating/loading APT.dll) with exception: ', e)

from time import sleep

class Rotation_Stage(object):
    '''
    Class to control a Thorlabs rotation stage using APT command library.
    Code is a simplified wrapper to thorlabs_apt package from PyPI.
    Defines the following user methods:
        rotate(speed=10.0) - starts constant angular velocity rotation (deg/sec) up to limit of motor

    Defines the following properties:
        identity - Get hardware information from motor controller. Returns tuple of strings containing hardware information.
        home - Send motor to home position. Returns True.
        angle - Property to Get or set motor position (in degrees, mod 360).
        stop - stop constant rotation. Returns True.
    '''
    def __init__(self, device_index=0):
        try:
            motor = apt_rs.list_available_devices()[device_index][1]
        except IndexError as e:
            raise RuntimeError('Cannot detect Thorlabs rotation stage controller. Check power and USB connection cable, then power-cycle the rotation stage controller to re-establish connection to PC.') from e
            
        self._motor = apt_rs.Motor(motor)
        self._acc, self._vmax =self._motor.get_velocity_parameter_limits()
        self._defaultv = 10.0
        self._defaultacc = 10.0
        self._motor.set_velocity_parameters(0.0, self._defaultacc, self._defaultv)

    @property
    def identity(self):
        '''
        Get hardware information from motor controller.
        Returns tuple of strings containing hardware information
        '''
        self._motor.identify()
        return self._motor.hardware_info

    @property
    def home(self):
        '''
        Send motor to home position.
        returns True
        '''
        self._motor.move_home(True)
        return True

    @property
    def angle(self):
        '''
        Property to Get or set motor position (in degrees, mod 360).
        '''
        return self._motor.position

    @angle.setter
    def angle(self, set_angle):
        self._motor.move_to(set_angle, True)

    def rotate(self, speed=10.0):
        '''
        Start rotation at constant angular velocity (degrees per second) set by speed (up to motor limit)
        returns True
        '''

        if (speed > self._vmax):
            speed = self._vmax
        #print('speed = {}').format(speed)
        self._motor.set_velocity_parameters(0.0, self._acc, float(speed))
        self._motor.move_velocity(2)
        return True

    @property
    def stop(self):
        '''
        Stop constant rotation.
        returns True
        '''
        self._motor.stop_profiled()
        sleep(1)
        self._motor.set_velocity_parameters(0.0, 10.0, 10.0)
        return True


if __name__ == '__main__':
    
     print(apt_rs.list_available_devices())
     
     rs = Rotation_Stage()
     print(rs.identity)
     
     rs.home
     print('homed')
     for i in range(90):
         rs.angle = i+1
         print(i+1)

[]


RuntimeError: Cannot detect Thorlabs rotation stage controller. Check power and USB connection cable, then power-cycle the rotation stage controller to re-establish connection to PC.